In [ ]:

import sys,os,datetime
import numpy as np
import pandas as pd
from binance.client import Client
import time
import progressbar
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
from binance.enums import *
import mpl_finance
# from BinanceKeys import BinanceKey1
# sys.path.append('C:\\Users\\Admin\\python-binance-master\\examples')

# os.chdir('C:/Users/Admin/python-binance-master/examples')
# sys.path.append('C:/Users/Admin/python-binance-master/examples')

import site
site.addsitedir(r'C:\\Users\\Admin\\python-binance-master\\examples')

import save_historical_data_Roibal
from BinanceKeys import BinanceKey1
# import technical_indicators

api_key = BinanceKey1['api_key']
api_secret = BinanceKey1['api_secret']

client = Client(api_key, api_secret)

# ================ Define variable values =================
watch_list = ['ETHBTC','ADABTC','EOSBTC','XRPBTC']

start_date = "1 month ago UTC"
kline = Client.KLINE_INTERVAL_5MINUTE
ping_interval = 120

ma_length = 60
# =========================================================

# Test Orders first to check

#Define Market Orders for ETH - Buy and sell
# buy_order_eth = client.order_market_buy(symbol='ETHBTC',quantity=0.01)
# sell_order_eth = client.order_market_sell(symbol='ETHBTC',quantity=0.01)

# ================== Main Function ====================
def run():
    # get system status
    #Create List of Crypto Pairs to Watch
#     list_of_symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT','BNBBTC', 'ETHBTC', 'LTCBTC']

    print("\n\n----------------------------------------------------------------------\n\n")
    print("Welcome to the Binance Bot Python Script\nCreated 2019 by Manoj Shenoy\n")
    print("You can also find this on github- https://www.github.com/manoj-shenoy")
    print("                                  https://www.github.com/stevens-morons")
    
    time.sleep(5)
#     try:
#         #Example Visualizations of Coins
        
#         for coin in watch_list:
            
#             save_historical_data_Roibal.save_historic_klines_csv(coin,
#                                                                  "1 month ago UTC", "now UTC",
#                                                                  Client.KLINE_INTERVAL_4HOUR)
#     except():
#         pass
    
    #Get Status of Exchange & Account
    try:
        status = client.get_system_status()
        print("\nExchange Status: ", status)

        #Account Withdrawal History Info
#         withdraws = client.get_withdraw_history()
#         print("\nClient Withdraw History: ", withdraws)

        #get Exchange Info
#         info = client.get_exchange_info()
#         print("\nExchange Info (Limits): ", info)
    
    except():
        pass

    # place a test market buy order, to place an actual order use the create_order function
    # if '1000 ms ahead of server time' error encountered, visit https://github.com/sammchardy/python-binance/issues/249
#     try:
#         order = client.create_test_order(
#             symbol='ETHBTC',
#             side=Client.SIDE_BUY,
#             type=Client.ORDER_TYPE_MARKET,
#             quantity=0.0001)
#     except:
#         print("\n \n \nATTENTION: NON-VALID CONNECTION WITH BINANCE \n \n \n")

    #Get Info about Coins in Watch List
    coin_prices(watch_list=watch_list)
    coin_tickers(watch_list=watch_list)
    
    # Get historical data using Klines DatFrame
#     data_eth = create_klines_df(symbol='ETHBTC',start_date=start_date,kline=kline)
    
    # Delete any old files
#     remove_files(csv='Binance_ETHBTC.txt')
#     remove_files(csv='Binance_ADABTC.txt')
#     remove_files(csv='Binance_EOSBTC.txt')
    
    # Save Historical Data for Coins
#     save_coin_files(start_date=start_date,kline=kline) # Currently ETHBTC, ADABTC, EOSBTC
    

#     data_eth = read_coin_files(csv='Binance_ETHBTC.txt')
    
#     print(data_eth.columns)
#     print('ETH Binance data below\n')
#     print(data_eth)
#     data_ada = read_coin_files(csv='Binance_ADABTC.txt')
#     data_eos = read_coin_files(csv='Binance_EOSBTC.txt')
    
    #for symbol in list_of_symbols:
    #    market_depth(symbol)
    
    # ===== Run all bots here ======
    mabandsbot(symbol='ETHBTC',ping_interval=ping_interval)
#     mabandsbot(historical_data=data_ada,symbol='ADABTC',wait_time=60)
#     mabandsbot(historical_data=data_eos,symbol='EOSBTC',wait_time=60)
    
    try:
        for symbol in watch_list:
            #get recent trades
            trades = client.get_recent_trades(symbol=symbol)
            print("\nRecent Trades: ", trades)
            print("Local Time: ", time.localtime())
            print("Recent Trades Time: ", convert_time_binance(trades[0]['time']))

            #get historical trades
            hist_trades = client.get_historical_trades(symbol=symbol)
            print("\nHistorical Trades: ", hist_trades)
            
            #get aggregate trades
            agg_trades = client.get_aggregate_trades(symbol=symbol)
            print("\nAggregate Trades: ", agg_trades)
    except:
        print('\n \n \nATTENTION: NON VALID CONNECTION WITH BINANCE \n \n \n')


def convert_time_binance(gt):
    #Converts from Binance Time Format (milliseconds) to time-struct
    #From Binance-Trader Comment Section Code
    #gt = client.get_server_time()
    print("Binance Time: ", gt)
    print(time.localtime())
    aa = str(gt)
    bb = aa.replace("{'serverTime': ","")
    aa = bb.replace("}","")
    gg=int(aa)
    ff=gg-10799260
    uu=ff/1000
    yy=int(uu)
    tt=time.localtime(yy)
    #print(tt)
    return tt


def market_depth(sym, num_entries=20):
    #Get market depth
    #Retrieve and format market depth (order book) including time-stamp
    i=0     #Used as a counter for number of entries
    print("Order Book: ", convert_time_binance(client.get_server_time()))
    depth = client.get_order_book(symbol=sym)
    print(depth)
    print(depth['asks'][0])
    bid_tot,ask_tot=0.0
    ask_price =[]
    ask_quantity = []
    bid_price = []
    bid_quantity = []
    place_order_ask_price,place_order_bid_price,max_order_ask,max_order_bid = 0

    print("\n", sym, "\nDepth     ASKS:\n")
    print("Price     Amount")
    for ask in depth['asks']:
        if i<num_entries:
            if float(ask[1])>float(max_order_ask):
                #Determine Price to place ask order based on highest volume
                max_order_ask=ask[1]
                place_order_ask_price=round(float(ask[0]),5)-0.0001
            #ask_list.append([ask[0], ask[1]])
            ask_price.append(float(ask[0]))
            ask_tot+=float(ask[1])
            ask_quantity.append(ask_tot)
            #print(ask)
            i+=1
    j=0     #Secondary Counter for Bids
    print("\n", sym, "\nDepth     BIDS:\n")
    print("Price     Amount")
    for bid in depth['bids']:
        if j<num_entries:
            if float(bid[1])>float(max_order_bid):
                #Determine Price to place ask order based on highest volume
                max_order_bid=bid[1]
                place_order_bid_price=round(float(bid[0]),5)+0.0001
            bid_price.append(float(bid[0]))
            bid_tot += float(bid[1])
            bid_quantity.append(bid_tot)
            #print(bid)
            j+=1
    return ask_price, ask_quantity, bid_price, bid_quantity, place_order_ask_price, place_order_bid_price
    #Plot Data

# def scalping_orders(coin, wait=1, tot_time=1):
#     #Function for placing 'scalp orders'
#     #Calls on Visualizing Scalping Orders Function
#     ap, aq, bp, bq, place_ask_order, place_bid_order, spread, proj_spread, max_bid, min_ask = visualize_market_depth(wait, tot_time, coin)
#     print("Coin: {}\nPrice to Place Ask Order: {}\nPrice to place Bid Order: {}".format(coin,place_ask_order, place_bid_order))
#     print("Spread: {} % Projected Spread {} %".format(spread, proj_spread))
#     print("Max Bid: {} Min Ask: {}".format(max_bid, min_ask))
#     #Place Orders based on calculated bid-ask orders if projected > 0.05% (transaction fee)
#    
#     """
#     if proj_spread > 0.05:
#         quant1=100          #Determine Code Required to calculate 'minimum' quantity
#         #Place Bid Order:
#         bid_order1 = client.order_limit_buy(
#             symbol=coin,
#             quantity=quant1,
#             price=place_bid_order)
#         #Place Ask Order
#         ask_order1 = client.order_limit_sell(
#             symbol=coin,
#             quantity=quant1,
#             price=place_ask_order)


# Place second order if current spread > 0.05% (transaction fee)



# def visualize_market_depth(wait_time_sec='1', tot_time='1', sym='ICXBNB', precision=5):
#     cycles = int(tot_time)/int(wait_time_sec)
#     start_time = time.asctime()
#     fig, ax = plt.subplots()
#     for i in range(1,int(cycles)+1):
#         ask_pri, ask_quan, bid_pri, bid_quan, ask_order, bid_order = market_depth(sym)

#         #print(ask_price)
#         plt.plot(ask_pri, ask_quan, color = 'red', label='asks-cycle: {}'.format(i))
#         plt.plot(bid_pri, bid_quan, color = 'blue', label = 'bids-cycle: {}'.format(i))

#         #ax.plot(depth['bids'][0], depth['bids'][1])
#         max_bid = max(bid_pri)
#         min_ask = min(ask_pri)
#         max_quant = max(ask_quan[-1], bid_quan[-1])
#         spread = round(((min_ask-max_bid)/min_ask)*100,5)   #Spread based on market
#         proj_order_spread = round(((ask_order-bid_order)/ask_order)*100, precision)
#         price=round(((max_bid+min_ask)/2), precision)
#         plt.plot([price, price],[0, max_quant], color = 'green', label = 'Price - Cycle: {}'.format(i)) #Vertical Line for Price
#         plt.plot([ask_order, ask_order],[0, max_quant], color = 'black', label = 'Ask - Cycle: {}'.format(i))
#         plt.plot([bid_order, bid_order],[0, max_quant], color = 'black', label = 'Buy - Cycle: {}'.format(i))
#         #plt.plot([min_ask, min_ask],[0, max_quant], color = 'grey', label = 'Min Ask - Cycle: {}'.format(i))
#         #plt.plot([max_bid, max_bid],[0, max_quant], color = 'grey', label = 'Max Buy - Cycle: {}'.format(i))
#         ax.annotate("Max Bid: {} \nMin Ask: {}\nSpread: {} %\nCycle: {}\nPrice: {}"
#                     "\nPlace Bid: {} \nPlace Ask: {}\n Projected Spread: {} %".format(max_bid, min_ask, spread, i, price, bid_order, ask_order, proj_order_spread),
#                     xy=(max_bid, ask_quan[-1]), xytext=(max_bid, ask_quan[0]))
#         if i==(cycles+1):
#             break
#         else:
#             time.sleep(int(wait_time_sec))
#     #end_time = time.asctime()
#     ax.set(xlabel='Price', ylabel='Quantity',
#        title='Binance Order Book: {} \n {}\n Cycle Time: {} seconds - Num Cycles: {}'.format(sym, start_time, wait_time_sec, cycles))
#     plt.legend()
#     plt.show()
#     return ask_pri, ask_quan, bid_pri, bid_quan, ask_order, bid_order, spread, proj_order_spread, max_bid, min_ask


def coin_prices(watch_list):
    #Will print to screen, prices of coins on 'watch list'
    #returns all prices
    prices = client.get_all_tickers()
    print("\nSelected (watch list) Ticker Prices: ")
    for price in prices:
        if price['symbol'] in watch_list:
            print(price)
    return prices


def coin_tickers(watch_list):
    # Prints to screen tickers for 'watch list' coins
    # Returns list of all price tickers
    tickers = client.get_orderbook_tickers()
    print("\nWatch List Order Tickers: \n")
    for tick in tickers:
        if tick['symbol'] in watch_list:
            print(tick)
    return tickers

def portfolio_management(deposit = '10000', withdraw=0, portfolio_amt = '0', portfolio_type='USDT', test_acct='True'):
    """The Portfolio Management Function will be used to track profit/loss of Portfolio in Any Particular Currency (Default: USDT)"""
    #Maintain Portfolio Statistics (Total Profit/Loss) in a file
    pass

# Determine position size of each asset
def position_sizing(symbol,percent_of_asset):
    asset_balance=client.get_asset_balance(asset=symbol)
    position_size=round(percent_of_asset * float(asset_balance['free']),4)
    return position_size

def create_klines_df(symbol,start_date,kline):
    header=['Time','Open','High','Low','Close']
    
    klines=client.get_historical_klines(symbol, kline, start_date, "now UTC")
    ohlc=[]
    for kline in klines:

        time1 = int(kline[0])
        open1 = float(kline[1])
        Low = float(kline[3])
        High = float(kline[2])
        Close = float(kline[4])
        Volume = float(kline[5])
#         format_kline = "{}, {}, {}, {}, {}, {}\n".format(time, open1, High, Low, Close, Volume)
        ohlc.append([time1, open1, High, Low, Close, Volume])
    
    klines=np.array(ohlc)

    klines_reqd=klines[:,0:5]

    klines_df=pd.DataFrame(klines_reqd,columns=header)
    print(klines_df.tail(5))
    return klines_df

def remove_files(csv):
    try:
        os.remove(csv)
        print('Old File removed!')
    
    except FileNotFoundError:
        print('The system cannot find the file specified')

# save ETH, ADA and EOS files by running save_historical_data
def save_coin_files(start_date,kline):
    for symbol in watch_list:
        save_historical_data_Roibal.save_historic_klines_csv(symbol, start_date, "now UTC", kline)    
        print('{}-Historical Data File saved '.format(symbol))


def read_coin_files(csv):
    read_data=pd.read_csv(csv, delimiter=',', encoding="utf-8-sig")
#     print(type(read_data))
    return read_data

# data = [data_eth,data_ada,data_eos]
     
#MACD, MACD Signal and MACD difference
def MABands(df, n):
    EMAHigh = pd.Series(df['High'].rolling(n).mean(),name='EMAHigh')
    EMALow = pd.Series(df['Low'].rolling(n).mean(),name='EMALow')

    df = df.join(EMAHigh)
    df = df.join(EMALow)

    return df

def vol_burst():
    pass        
        
def progress():
    bar = progressbar.ProgressBar(maxval=20, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i in range(20):
        bar.update(i+1)
        time.sleep(0.1)
    bar.finish()

n=126    
def mabandsbot(symbol,ping_interval=ping_interval):
        
    
    # Initialise position
    position=0
    percent_of_asset=float(0.9)
    trade_size = position_sizing(symbol=symbol[:3],
                                 percent_of_asset=percent_of_asset)
    num_cycles = 1e+15
    for i in range(1,int(num_cycles)): 
    #     position=0
        # Delete any old files
#         remove_files(csv='Binance_ETHBTC.txt')
#         remove_files(csv='Binance_ADABTC.txt')
#         remove_files(csv='Binance_EOSBTC.txt')
    
#         Save Historical Data for Coins
#         save_coin_files(start_date=start_date,kline=kline) # Currently ETHBTC, ADABTC, EOSBTC
        
        data_eth = create_klines_df(symbol='ETHBTC',start_date=start_date,kline=kline)
        
        ma_bands=MABands(df=data_eth,n=ma_length)
        ma_high = ma_bands['EMAHigh']
        ma_low = ma_bands['EMALow']
        close = data_eth['Close']
        
        print('Previous Close | Previous MA High | Previous MA Low' )
        print('    {}           {}           {}'.format(round(close.iloc[-3],6),
                                                        round(ma_high.iloc[-3],6),
                                                        round(ma_low.iloc[-3],6)))
        
        print('Recent Close | Recent MA High | Recent MA Low' )
        print('    {}         {}        {}'.format(round(close.iloc[-2],6),
                                                           round(ma_high.iloc[-2],6),
                                                           round(ma_low.iloc[-2],6)))
                
        buy_condition = (close.iloc[-2] > ma_high.iloc[-2]) and (close.iloc[-3] < ma_high.iloc[-3]) and (position==0)
        print('Buy Condition ', buy_condition)
        
        sell_condition = (close.iloc[-2] < ma_low.iloc[-2]) and (close.iloc[-3] > ma_low.iloc[-3]) and (position==0)
        print('Sell Condition ', sell_condition)
        
        print ('\nRunning Epoch {}/{}'.format(i,int(num_cycles)))
        progress()
        
        if (close.iloc[-2] > ma_high.iloc[-2] and
            close.iloc[-3] < ma_high.iloc[-3] and
            position==0):
            print('Buy Condition Triggered\n')
#             print('{}'.format(symbol)+' Bought at '+ '{}'.format(market_depth(symbol,num_entries=20)[0]))
            client.order_market_buy(symbol=symbol,
                                    quantity=trade_size)
            position+=trade_size # change this to actual position size
            print('{} Qty of {} Bought at {}'.format(trade_size,
                                                     symbol,
                                                     client.get_my_trades(symbol=symbol)[-1]['price']))
        
        elif (close.iloc[-2] < ma_low.iloc[-2] and
              close.iloc[-3] > ma_low.iloc[-3] and
              position==0):
            print('Sell Condition Triggered\n')
#             print('{}'.format(symbol)+' Sold at '+ '{}'.format(market_depth(symbol,num_entries=20)[2]))
            client.order_market_sell(symbol=symbol,
                                     quantity=trade_size)
            position-=trade_size
            print('{} Qty of {} Sold at {}'.format(trade_size,
                                                   symbol,
                                                   client.get_my_trades(symbol=symbol)[-1]['price']))

        if (position > 0 and close.iloc[-2] < ma_low.iloc[-2] and
            close.iloc[-3] > ma_low.iloc[-3]):
            print('Stop-Loss of Buy Position Triggered\n')
#             print('{}'.format(symbol)+' Sold at '+ '{}'.format(market_depth(symbol,num_entries=20)[2]))
            client.order_market_sell(symbol=symbol,
                                     quantity=position)
            position=0
            print('{} Qty of {} Stop-Loss- Sell triggered at {}'.format(position,
                                                                        symbol,
                                                                        client.get_my_trades(symbol=symbol)[-1]['price']))
        
        elif (position < 0 and close.iloc[-2] > ma_high.iloc[-2] and
            close.iloc[-3] < ma_high.iloc[-3]):
            print('Stop-Loss of Sell Position Triggered\n')
#             print('{}'.format(symbol)+' Bought at '+ '{}'.format(market_depth(symbol,num_entries=20)[0]))
            client.order_market_buy(symbol=symbol,
                                    quantity=abs(position))
            position=0
            print('{} Qty of {} Stop-Loss- Sell triggered at {}'.format(position,
                                                                        symbol,
                                                                        client.get_my_trades(symbol=symbol)[-1]['price']))

        else:
            print("No Trading Signal Triggered")
        
        print('Executed Time :', datetime.datetime.now())
        time.sleep(ping_interval)

"""
#trade aggregator (generator)
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', start_str='30 minutes ago UTC')
# iterate over the trade iterator
for trade in agg_trades:
    pass
    #print(trade)
    # do something with the trade data

# convert the iterator to a list
# note: generators can only be iterated over once so we need to call it again
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', start_str='30 minutes ago UTC')
agg_trade_list = list(agg_trades)

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")
#for kline in klines:
    #print(kline)
"""


if __name__ == "__main__":
    run()




----------------------------------------------------------------------


Welcome to the Binance Bot Python Script
Created 2019 by Manoj Shenoy

You can also find this on github- https://www.github.com/manoj-shenoy
                                  https://www.github.com/stevens-morons

Exchange Status:  {'msg': 'normal', 'status': 0}

Selected (watch list) Ticker Prices: 
{'symbol': 'ETHBTC', 'price': '0.01735900'}
{'symbol': 'EOSBTC', 'price': '0.00034480'}
{'symbol': 'XRPBTC', 'price': '0.00002616'}
{'symbol': 'ADABTC', 'price': '0.00000462'}

Watch List Order Tickers: 

{'symbol': 'ETHBTC', 'bidPrice': '0.01735900', 'bidQty': '1.12300000', 'askPrice': '0.01736200', 'askQty': '0.53300000'}
{'symbol': 'EOSBTC', 'bidPrice': '0.00034480', 'bidQty': '397.98000000', 'askPrice': '0.00034510', 'askQty': '12.06000000'}
{'symbol': 'XRPBTC', 'bidPrice': '0.00002613', 'bidQty': '11992.00000000', 'askPrice': '0.00002615', 'askQty': '7182.00000000'}
{'symbol': 'ADABTC', 'bidPrice': '0.00000462'

              Time      Open      High       Low     Close
8635  1.577292e+12  0.017335  0.017374  0.017324  0.017366
8636  1.577293e+12  0.017368  0.017378  0.017360  0.017370
8637  1.577293e+12  0.017370  0.017380  0.017360  0.017370
8638  1.577293e+12  0.017372  0.017376  0.017355  0.017372
8639  1.577294e+12  0.017373  0.017373  0.017353  0.017359
Previous Close | Previous MA High | Previous MA Low
    0.01737           0.017301           0.017267
Recent Close | Recent MA High | Recent MA Low
    0.017372         0.017302        0.017269
Buy Condition  False
Sell Condition  False

Running Epoch 1/1000000000000000


[========================================================================] 100%


No Trading Signal Triggered
Executed Time : 2019-12-25 12:08:12.477313


              Time      Open      High       Low     Close
8635  1.577293e+12  0.017368  0.017378  0.017360  0.017370
8636  1.577293e+12  0.017370  0.017380  0.017360  0.017370
8637  1.577293e+12  0.017372  0.017376  0.017355  0.017372
8638  1.577294e+12  0.017373  0.017373  0.017340  0.017342
8639  1.577294e+12  0.017342  0.017351  0.017342  0.017348
Previous Close | Previous MA High | Previous MA Low
    0.017372           0.017302           0.017269
Recent Close | Recent MA High | Recent MA Low
    0.017342         0.017302        0.017271
Buy Condition  False
Sell Condition  False

Running Epoch 2/1000000000000000


...

[========================================================================] 100%


No Trading Signal Triggered
Executed Time : 2019-12-25 13:27:03.188727


              Time      Open      High       Low     Close
8635  1.577297e+12  0.017379  0.017379  0.017367  0.017375
8636  1.577297e+12  0.017374  0.017382  0.017370  0.017379
8637  1.577298e+12  0.017379  0.017390  0.017371  0.017377
8638  1.577298e+12  0.017376  0.017385  0.017364  0.017377
8639  1.577298e+12  0.017378  0.017399  0.017377  0.017396
Previous Close | Previous MA High | Previous MA Low
    0.017377           0.017323           0.017296
Recent Close | Recent MA High | Recent MA Low
    0.017377         0.017324        0.017297
Buy Condition  False
Sell Condition  False

Running Epoch 37/1000000000000000


[========================================================================] 100%


No Trading Signal Triggered
Executed Time : 2019-12-25 13:29:17.764093


In [41]:
klines=client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "30 Dec, 2017", "1 Jan, 2018")
ohlc=[]
for kline in klines:
    #print(kline)
    time1 = int(kline[0])
    open1 = float(kline[1])
    Low = float(kline[3])
    High = float(kline[2])
    Close = float(kline[4])
    Volume = float(kline[5])

ohlc.append([time1, open1, Close, High, Low, Volume])

print(ohlc)

[[1514764800000, 0.053586, 0.05375, 0.054042, 0.053474, 6805.697]]


In [63]:
import numpy as np
header=['Time','Open','High','Low','Close']
klines=client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "30 Dec, 2017", "1 Jan, 2018")
klines=np.array(klines)

klines_1=klines[:,0:5]
# print(klines_1)
klines_2=pd.DataFrame(klines_1,columns=header)
# print(klines_2)

In [37]:
client.get_historical_klines("ETHBTC",
                             Client.KLINE_INTERVAL_4HOUR,
                             "1 Month ago UTC - 5", "now UTC - 5")[-2]

[1577232000000,
 '0.01760200',
 '0.01762200',
 '0.01730800',
 '0.01739000',
 '16841.08400000',
 1577246399999,
 '293.33468444',
 15641,
 '6844.20100000',
 '119.23647551',
 '0']